<a href="https://colab.research.google.com/github/Beloveddream/RS/blob/main/team_steam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**team_cluster**

In [2]:
# coding: utf-8
from sklearn.cluster import KMeans
from sklearn import preprocessing
import pandas as pd
import numpy as np

In [5]:
# 数据加载
data = pd.read_csv('/content/gdrive/My Drive/les1/team/team_cluster_data.csv', encoding='gbk')
train_x = data[["2019国际排名","2018世界杯排名","2015亚洲杯排名"]]
kmeans = KMeans(n_clusters=3)
# 规范化到 [0,1] 空间
min_max_scaler=preprocessing.MinMaxScaler()
train_x=min_max_scaler.fit_transform(train_x)
print(train_x)

[[0.3        0.71428571 0.375     ]
 [0.2        0.         0.25      ]
 [0.20769231 0.11428571 0.0625    ]
 [0.         0.08571429 0.3125    ]
 [0.25384615 0.31428571 0.5625    ]
 [0.43846154 0.71428571 0.1875    ]
 [0.51538462 0.71428571 0.75      ]
 [0.36153846 0.71428571 0.3125    ]
 [0.41538462 0.71428571 0.4375    ]
 [0.67692308 0.71428571 1.        ]
 [0.52307692 1.         1.        ]
 [0.40769231 1.         0.6875    ]
 [0.58461538 1.         0.8125    ]
 [1.         1.         1.        ]
 [0.04615385 0.42857143 0.        ]
 [0.32307692 0.71428571 1.        ]
 [0.64615385 1.         0.5       ]
 [0.96923077 1.         0.875     ]]


In [6]:
# kmeans 算法
kmeans.fit(train_x)
predict_y = kmeans.predict(train_x)
# 合并聚类结果，插入到原数据中
result = pd.concat((data,pd.DataFrame(predict_y)),axis=1)
result.rename({0:u'聚类结果'},axis=1,inplace=True)
print(result)
#将结果导出到CSV文件中
#result.to_csv("team_cluster_result.csv")

        国家  2019国际排名  2018世界杯排名  2015亚洲杯排名  聚类结果
0       中国        73         40          7     0
1       日本        60         15          5     2
2       韩国        61         19          2     2
3       伊朗        34         18          6     2
4       沙特        67         26         10     2
5      伊拉克        91         40          4     0
6      卡塔尔       101         40         13     1
7      阿联酋        81         40          6     0
8   乌兹别克斯坦        88         40          8     0
9       泰国       122         40         17     1
10      越南       102         50         17     1
11      阿曼        87         50         12     1
12      朝鲜       110         50         14     1
13      印尼       164         50         17     1
14      澳洲        40         30          1     2
15     叙利亚        76         40         17     1
16      约旦       118         50          9     0
17     科威特       160         50         15     1


In [7]:
import numpy as np 
import pandas as pd 
import random

In [9]:
path = '/content/gdrive/My Drive/les1/steam/steam-200k.csv'
df = pd.read_csv(path, header = None, names = ['UserID', 'Game', 'Action', 'Hours', 'Not Needed'])
# 数据探索
print('显示前5条数据')
print(df.head())
print('显示数据大小')
print(df.shape)
# 创建Hours_Played字段，替代原有的Action和Hours，0表示仅购买，大于0表示购买且游戏时长
df['Hours_Played'] = df['Hours'].astype('float32')
# 如果字段Action=purchase，并且Hours=1.0，将设置Hours_Played=0
df.loc[(df['Action'] == 'purchase') & (df['Hours'] == 1.0), 'Hours_Played'] = 0
print(df['Hours_Played'])
print('增加了Hours_Played字段后，数据大小')
print(df.shape)

显示前5条数据
      UserID                        Game    Action  Hours  Not Needed
0  151603712  The Elder Scrolls V Skyrim  purchase    1.0           0
1  151603712  The Elder Scrolls V Skyrim      play  273.0           0
2  151603712                   Fallout 4  purchase    1.0           0
3  151603712                   Fallout 4      play   87.0           0
4  151603712                       Spore  purchase    1.0           0
显示数据大小
(200000, 5)
0           0.0
1         273.0
2           0.0
3          87.0
4           0.0
          ...  
199995      1.5
199996      0.0
199997      1.5
199998      0.0
199999      1.4
Name: Hours_Played, Length: 200000, dtype: float32
增加了Hours_Played字段后，数据大小
(200000, 6)


In [10]:
# 对数据从小到大进行排序, df下标也会发生变化
df.UserID = df.UserID.astype('int')
df = df.sort_values(['UserID', 'Game', 'Hours_Played'], ascending=True)

# 删除重复项，并保留最后一项出现的项（因为最后一项是用户游戏时间，第一项为购买）
clean_df = df.drop_duplicates(['UserID', 'Game'], keep = 'last')
# 去掉不用的列：Action, Hours, Not Needed
clean_df = clean_df.drop(['Action', 'Hours', 'Not Needed'], axis = 1)
print('删除重复项后的数据集：')
print(clean_df)
print(clean_df.head(0))

# 探索下数据集的特征
n_users = len(clean_df.UserID.unique())
n_games = len(clean_df.Game.unique())
print('数据集中包含了 {0} 玩家，{1} 游戏'.format(n_users, n_games))

# 矩阵的稀疏性
sparsity = clean_df.shape[0] / float(n_users * n_games)
print('用户行为矩阵的稀疏性（填充比例）为{:.2%} '.format(sparsity))

删除重复项后的数据集：
           UserID                          Game  Hours_Played
65430        5250                   Alien Swarm           4.9
65424        5250               Cities Skylines         144.0
65435        5250                Counter-Strike           0.0
65436        5250         Counter-Strike Source           0.0
65437        5250                 Day of Defeat           0.0
...           ...                           ...           ...
18803   309626088  Age of Empires II HD Edition           6.7
170024  309812026  Counter-Strike Nexon Zombies           0.0
170025  309812026                     Robocraft           0.0
10222   309824202                        Dota 2           0.7
129085  309903146                        Dota 2           0.2

[128804 rows x 3 columns]
Empty DataFrame
Columns: [UserID, Game, Hours_Played]
Index: []
数据集中包含了 12393 玩家，5155 游戏
用户行为矩阵的稀疏性（填充比例）为0.20% 
